In [1]:
"""Created on Monday Jan 5 14:03:00 
2019 Group 2
@authors: Sibel Yuksel, Marianne Manson (v1)"""

'Created on Monday Jan 5 14:03:00 \n2019 Group 2\n@authors: Sibel Yuksel'

In [2]:
!pip install textblob

In [15]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup

In [2]:
df = pd.DataFrame(columns=['art_content','art_content_html','art_extract_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag']  )
df

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag


In [27]:
url = 'https://www.parlonsrh.com/raisons-utiliser-lintelligence-artificielle-dans-gestion-gpec/'
url2 = "https://www.parlonsrh.com/actus/2020-une-annee-pas-comme-les-autres/"


def get_Title(url : str) -> str :
    """
    This function extracts the title of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    html_titre = html_soup.title
    titre = html_titre.get_text()
    return titre
    
def get_Time(url : str) -> str :
    """
    This function extracts the time of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    date = html_soup.find("span", {"class":"date updated value-title"})["title"]
    return date

def get_lang(titre : str) -> str : 
    """
    This function analyse the webpage/article language
    """
    a = TextBlob(titre)
    langue = a.detect_language()
    return langue

def get_Img(url : str) -> str :
    """
    This function extracts the introducing picture of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    img = html_soup.find('meta', {'property':'og:image'})['content'] 
    return img


def art_content_html(titre : str) -> str :
    """
    This function extracts the html content of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    paragraphe = html_soup.find_all('p')
    content_html = " ".join([str(x) for x in paragraphe])
    return content_html
    
def art_content(titre : str) -> str :
    """
    This function extracts the clean content of the webpage/article without 
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    paragraphe = html_soup.find_all('p')
    content = " ".join([x.text for x in paragraphe])
    return content

def get_Tag(url : str) -> str :
    """
    This function extracts different tags of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    html_tag = html_soup.find_all("meta",{'property':"article:tag"})
    Tags = []
    for i in html_tag:
        tag_i = i['content']
        Tags.append(tag_i)
    return Tags

def get_Author(url : str) -> str :
    """
    This function extracts the name of the author of the webpage/article
    """
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    author = html_soup.find("span",{"class":"fn"}).get_text()
    if author[11:29]=='La Team Parlons RH':
        return author[11:29] 
    else:
        return author[11:34]

def remplir_dataFrame(url):
    """
    This function add different information of the webpage/article to the dataFrame
    """
    df.loc[0, 'src_type'] = 'xpath_source'
    df.loc[0, 'src_url'] = 'https://www.parlonsrh.com/'
    df.loc[0, 'src_name'] = 'parlonsrh'
    df.loc[0, 'art_url'] = url
    
    df.loc[0, 'art_auth'] = get_Author(url)
    tag = get_Tag(url)
    if tag is None or tag==[ ]:
        tag = 'no data'
    df.loc[0, 'art_tag'] = tag
    df.loc[0, 'art_content_html'] = art_content_html(url)
    df.loc[0, 'art_content'] = art_content(url)
    time = get_Time(url)
    if time is None or time==[ ]:
        time = 'no data'
    df.loc[0, 'art_extract_datetime'] = time
    img=get_Img(url)
    if img is None:
        df.loc[0, 'src_img'] = 'no data'
    else:
        df.loc[0, 'src_img'] = img
    titre = get_Title(url)
    df.loc[0, 'art_title'] = titre
    langue = get_lang(titre)
    df.loc[0, 'art_lang'] = langue
    return df


remplir_dataFrame(url)

,art_content,art_content_html,art_extract_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,La bonne connaissance des besoins de l’entrepr...,<p><strong>La bonne connaissance des besoins d...,26 Mai 2020,fr,Les raisons d'utiliser l’intelligence artifici...,https://www.parlonsrh.com/raisons-utiliser-lin...,parlonsrh,xpath_source,https://www.parlonsrh.com/,https://www.parlonsrh.com/wp-content/uploads/2...,Baptiste Julien Blandet,"[Big data, Compétences, Transformation rh]"


In [18]:
#url : 'https://www.parlonsrh.com/raisons-utiliser-lintelligence-artificielle-dans-gestion-gpec/'
def scraping(url):
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    #content
    paragraphe = html_soup.find_all('p')
    content = " ".join([x.text for x in paragraphe])
    #content_html
    content_html = " ".join([str(x) for x in paragraphe])
    #time
    time = html_soup.find("span", {"class":"date updated value-title"})["title"]
    if time is None or time==[ ]:
        time = 'no data'
    else:
        trans_month = {'01':['janvier'], 
             '02':['février'],
             '03':['mars'],
             '04':['avril'],
             '05':['mai'],
             '06':['juin'],
             '07':['juillet'],
             '08':['août'],
             '09':['septembre'],
             '10':['octobre'],
             '11':['novembre'],
             '12':['décembre']}
        date_tab = time.split(" ")
        day = date_tab[0]
        month = date_tab[1]
        for m in trans_month:
            if month.lower() in trans_month[m]:
                month = m
        year = date_tab[2]
        time = datetime.datetime(int(year), int(month), int(day))
    #title
    html_title = html_soup.title
    title = html_title.get_text()
    #img
    img = html_soup.find('meta', {'property':'og:image'})['content']
    if img is None:
        img = 'no_data'
    #author
    author = html_soup.find("span",{"class":"fn"}).get_text()
    if author[11:29]=='La Team Parlons RH':
        author = author[11:29] 
    else:
        author = author[11:34]
    #tag
    html_tag = html_soup.find_all("meta",{'property':"article:tag"})
    tags = []
    for i in html_tag:
        tag_i = i['content']
        tags.append(tag_i)
    if tags is None or tags==[ ]:
        tags = 'no data'
    new_row = {'art_content': content ,
               'art_content_html': content_html ,
               'art_published_datetime': time ,
               'art_lang': 'fr' , 
               'art_title' : title , 
               'art_url' : url ,
               'src_name' : 'parlonsrh' ,
               'src_type' : 'xpath_source' ,
               'src_url' : 'https://www.parlonsrh.com/' ,
               'src_img' : img ,
               'art_auth': author,
               'art_tag': tags}
    return new_row

In [19]:
#Test
df = pd.DataFrame(columns=['art_content','art_content_html','art_published_datetime','art_lang','art_title','art_url','src_name','src_type','src_url','src_img','art_auth','art_tag'])
new_row = scraping('https://www.parlonsrh.com/raisons-utiliser-lintelligence-artificielle-dans-gestion-gpec/')
df_test = df.append(new_row, ignore_index=True)
df_test

,art_content,art_content_html,art_published_datetime,art_lang,art_title,art_url,src_name,src_type,src_url,src_img,art_auth,art_tag
0,La bonne connaissance des besoins de l’entrepr...,<p><strong>La bonne connaissance des besoins d...,2020-05-26,fr,Les raisons d'utiliser l’intelligence artifici...,https://www.parlonsrh.com/raisons-utiliser-lin...,parlonsrh,xpath_source,https://www.parlonsrh.com/,https://www.parlonsrh.com/wp-content/uploads/2...,Baptiste Julien Blandet,"[Big data, Compétences, Transformation rh]"
